In [2]:
import pandas as pd
from preprocess import prepare_df
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function
from sklearn.metrics import r2_score
import numpy as np

In [3]:
data = pd.read_csv('../lstm/utils/df_imputed.csv', index_col=0).drop(columns=['date'])

In [4]:
df_sensor_s, df_sensor_t, df_gpp_s, df_gpp_t, df_domain_s, df_domain_t, masks = prepare_df(data)

In [5]:
DEVICE = torch.device("cuda:" + str(2))

In [6]:
# Part 1: feature extraction

class ConvFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, bidirectional):
        super().__init__()
        self.conv_blocks = nn.Sequential(
             nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=bidirectional)
        )
    def forward(self, x):
        out, (h,c) = self.conv_blocks(x)
        return out.squeeze()

In [7]:
# top branch: regressor 
class RecurrentRegressor(nn.Module):
    # gets a sequence of shape (seq len, batch size, n_channels)
    def __init__(self, input_dim, hidden_dim, num_layers, bidirectional):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if self.bidirectional else 1

        self.rnn = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=bidirectional)
        self.fc = nn.Sequential(
            nn.Linear(self.num_directions * hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        output = self.fc(x) #  seq_len, batch, 1
        return output

In [8]:
# one domain discriminator for every source domain
class DomainClassifier(nn.Module):
    # gets a sequence of shape (seq len, batch size, n_channels) --> (batch_size, seq_len * n_channels)
    def __init__(self, n_channels):
        super().__init__()
        self.n_channels = n_channels
        self.fc = nn.Sequential(
            nn.Linear(self.n_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        # x: (seq len, batch size, n_channels)
        # average over the seq_len dim
        x = torch.mean(x, dim=0) # x: (batch_size, n_channels)
        y_pred = self.fc(x)
        return y_pred

In [9]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [10]:
class Model(nn.Module):
    def __init__(self, extractor, regressor, classifiers):
        super().__init__()
        self.extractor = extractor
        self.regressor = regressor
        self.classifiers = classifiers

    def forward(self, s, t, alpha):
        s = s.permute(0, 1, 2)
        t = t.permute(0, 1,2)
        
        s_features = self.extractor(s)
        t_features = self.extractor(t)
        
       


        y_pred = self.regressor(s_features)
        
        rev_s_features = ReverseLayerF.apply(s_features, alpha)
        rev_t_features = ReverseLayerF.apply(t_features, alpha)
    
        s_preds = []
        t_preds = []
        
        for classifier in self.classifiers:
            s_domain_pred = classifier(rev_s_features)
            t_domain_pred = classifier(rev_t_features)
            s_preds.append(s_domain_pred)
            t_preds.append(t_domain_pred)

        return y_pred, s_preds, t_preds

In [11]:
def loss_fn(source_domains, target_domains, y, source_preds, target_preds, y_pred, mu):
    reg_loss = nn.MSELoss()(y_pred, y)
    domain_losses = []
    for i in range(9):
        loss2 = nn.BCELoss()(source_preds[i], source_domains)
        loss3 = nn.BCELoss()(target_preds[i], target_domains)
        domain_losses.append(loss2 + loss3)
    domain_loss = mu * torch.mean(torch.tensor(domain_losses))
    loss = reg_loss + domain_loss
    return loss, domain_loss, reg_loss

In [12]:
unwanted = [0, 1, 4, 5, 6, 8]
  
for ele in sorted(unwanted, reverse = True): 
    del df_sensor_s[ele]
    del df_gpp_s[ele]

In [15]:
len(df_sensor_s)

3

In [16]:
DOMAINS = 3
feature_extractor = ConvFeatureExtractor(11, 256, 1, False)
regressor = RecurrentRegressor(128, 256, 1, False)
classifiers = []
for t in range(DOMAINS):
    classifiers.append(DomainClassifier(256).to(DEVICE))

model = Model(feature_extractor, regressor, classifiers).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
import pdb
EPOCHS = 1000
alpha = 0.1
mu= 1
for epoch in range(EPOCHS):
    train_loss = 0.0
    train_reg_loss = 0.0
    train_domain_loss = 0.0
    train_r2 = 0.0
    model.train()
    for t in range(len(df_sensor_t)):
        domain_losses = []
        x_t = torch.FloatTensor(df_sensor_t[t].values).unsqueeze(1).to(DEVICE)
        y_t = torch.FloatTensor(df_gpp_t[t].values).to(DEVICE)
        domain_t  = torch.FloatTensor([1]).to(DEVICE)
        for k in range(DOMAINS):
          s_k = np.random.choice(range(len(df_sensor_s[k])))
          x_s  = torch.FloatTensor(df_sensor_s[k][s_k].values).unsqueeze(1).to(DEVICE)
          y_s  = torch.FloatTensor(df_gpp_s[k][s_k].values).to(DEVICE)
          domain_s = torch.FloatTensor([0]).to(DEVICE)
        
          y_pred, s_preds, t_preds = model(x_s, x_t, alpha)
          y_pred = y_pred.squeeze()
          # Get loss and update
          optimizer.zero_grad()

          loss = loss_fn(domain_s, domain_t, y_s, s_preds, t_preds, y_pred, 1)
          domain_losses.append(loss)
          train_r2 += r2_score(y_true=y_s.detach().cpu(), y_pred=y_pred.detach().cpu())
        
        worst_class = torch.tensor(domain_losses).argmin(axis=0)[1]
        worst_reg = torch.tensor(domain_losses).argmax(axis=0)[2]
        loss = domain_losses[worst_class][1] + mu * domain_losses[worst_reg][2]
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_domain_loss += domain_losses[worst_class][1].item()
        train_reg_loss += domain_losses[worst_reg][2].item()

    
    train_loss /= len(df_sensor_t)
    train_domain_loss /=  len(df_sensor_t)
    train_reg_loss /= len(df_sensor_t)
    train_r2 /= len(df_sensor_t)*DOMAINS
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.4f} | Reg loss: {train_reg_loss:.4f} | Domain loss: {train_domain_loss:.4f} | R2: {train_r2:.4f}")
        
        


IndexError: list index out of range

In [36]:
y_pred, s_preds, t_preds = model(x_s, x_t, alpha)
y_pred = y_pred.squeeze()





In [59]:
torch.tensor(domain_losses)

tensor([[2.4171, 1.3895, 1.0276],
        [2.4161, 1.3892, 1.0269],
        [2.4172, 1.3896, 1.0276]])

In [38]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=y_s.detach().cpu(),
                    mode='markers',
                    name='gt', 
                    marker=dict(size=3)))

fig.add_trace(go.Scatter(y=y_pred.detach().cpu(),
                    mode='markers',
                    name='pred', 
                    marker=dict(size=3)))

In [27]:
aaa

NameError: name 'aaa' is not defined